In [1]:
import pygeos
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.colors as colors
import datetime

In [2]:
%%time
consumption= pd.read_pickle('./data/processed/merged_smoothed_monthly_by_meternumber_consumerid_march032021.pck').rename(
    columns={'meter_serial_number':'Meter_Numb'}) # customer consumption data
# meta data
meta_df = pd.read_pickle('./data/processed/REG_metadata_3_Nov.pck') # customer_details
# split column and add new columns to df, add meter_numb column
a = meta_df['meter_serial_number_consumer_id'].str.split('_', expand=True)
meta_df['Meter_Numb'] = a[0]
meta_df['customer_ID'] = a[1]

# convert column 'Meter_Numb' to float
meta_df['Meter_Numb'] = pd.to_numeric(meta_df['Meter_Numb'],errors='coerce') 
meta_df['customer_ID'] = pd.to_numeric(meta_df['customer_ID'],errors='coerce')
meta_df = meta_df[['meter_serial_number_consumer_id', 'vending_category_name',
       'district_id', 'district', 'installation_date','eguide_categories', 'Meter_Numb']]

# convert to datetime
meta_df['installation_date'] = pd.to_datetime(meta_df.installation_date).dt.date

# Reliability data
mv_outages_2020 = pd.read_excel('./data/raw/reg_reliability/MV_Outages_2020.xlsx',
                                sheet_name='Detailed',header=3)
mv_outages_2019 = pd.read_excel('./data/raw/reg_reliability/MV_Outages_2016-2019.xlsx',
                                sheet_name='2019',header=2)
mv_outages_2018 = pd.read_excel('./data/raw/reg_reliability/MV_Outages_2016-2019.xlsx',
                                sheet_name='2018',header=1)
mv_outages_2017 = pd.read_excel('./data/raw/reg_reliability/MV_Outages_2016-2019.xlsx',
                                sheet_name='2017',header=2)
mv_outages_2016 = pd.read_excel('./data/raw/reg_reliability/MV_Outages_2016-2019.xlsx',
                                sheet_name='2016',header=1)
mv_outages_2016['Time(h)']=pd.to_timedelta(mv_outages_2016['Duration'].astype(str),
                                           errors='coerce').dt.total_seconds()/3600


# remove unnecessary columns 
mv_outages_2020 = mv_outages_2020.iloc[:,1:21] 
mv_outages_2019 = mv_outages_2019.iloc[:,1:15]
mv_outages_2018 = mv_outages_2018.iloc[:,1:15]
mv_outages_2017 = mv_outages_2017.iloc[:,1:]
mv_outages_2016 = mv_outages_2016.iloc[:,1:]


# to lower case
reliablity_dfs = [mv_outages_2020, mv_outages_2019, mv_outages_2018, mv_outages_2017, mv_outages_2016]
cols = ['Feeder','Substation Name','Hubs', 'Substation']
for num,df in enumerate(reliablity_dfs):
    for column in cols:
        try:
            df.loc[:,column] = df[column].str.lower()
            df.loc[:,column] = df[column].str.strip()
        except KeyError:
            _ = (f'{column} absent in dataframe')
##

# meter GPS
meter_gps = gpd.read_file('./data/raw/reg_gis/customer_locations_w_districts.geojson')

# lines
lines = gpd.read_file('./data/raw/reg_gis/LV_lines/lv_lines.shp')
cols = ['Feeder_Nam','Substation','Branch_Nam']
for column in cols:
    lines.loc[:,column] = lines[column].str.lower()
    lines.loc[:,column] = lines[column].str.strip()

# Use projected CRS
meter_gps = meter_gps.to_crs('epsg:32735')
lines = lines.to_crs('epsg:32735')
meter_cols = ['Meter_Numb','Transforme','MV_Feeder_',
        'Branch_Nam', 'Connection', 'longitude',
       'latitude', 'District', 'geometry']
lines_cols = ['OBJECTID','Substation','Feeder_Nam','Branch_Nam','geometry']

# merge meters with feeder lines
meter_line_dist = 80
meter_lines = meter_gps[meter_cols].sjoin_nearest(lines[lines_cols],
                       how='left', distance_col='distance_to_line', max_distance=meter_line_dist)

# meters with consumption
meter_numbers = meta_df.Meter_Numb.unique().tolist()
meter_lines_w_cons = meter_lines[meter_lines['Meter_Numb'].isin(meter_numbers)]

# add installation dates
meter_lines_w_cons = meter_lines_w_cons.merge(meta_df[['meter_serial_number_consumer_id','Meter_Numb','eguide_categories']],
                        how='left',on='Meter_Numb').drop('index_right', axis=1)

# Feeder reliability
feeder_reliability_2020 = mv_outages_2020.groupby(['Hubs','Feeder'])['Time(h)'].agg(saifi_2020='count',
                                                                  saidi_2020='sum').reset_index().rename(
    columns={'Feeder':'Feeder_Nam','Hubs':'Hubs_2020'})

feeder_reliability_2019 = mv_outages_2019.groupby(['Hubs','Feeder'])['Time(h)'].agg(saifi_2019='count',
                                                                  saidi_2019='sum').reset_index().rename(
    columns={'Feeder':'Feeder_Nam','Hubs':'Hubs_2019'})

feeder_reliability_2018 = mv_outages_2018.groupby(['Hubs','Feeder'])['Time(h)'].agg(saifi_2018='count',
                                                                  saidi_2018='sum').reset_index().rename(
    columns={'Feeder':'Feeder_Nam','Hubs':'Hubs_2018'})

feeder_reliability_2017 = mv_outages_2017.groupby(['Feeder'])['Time(h)'].agg(saifi_2017='count',
                                                                  saidi_2017='sum').reset_index().rename(
    columns={'Feeder':'Feeder_Nam'})

feeder_reliability_2016 = mv_outages_2016.groupby(['Feeder'])['Time(h)'].agg(saifi_2016='count',
                                                                  saidi_2016='sum').reset_index().rename(
    columns={'Feeder':'Feeder_Nam'})

# Merge dataframes
meter_lines_w_cons_w_reliabilty = meter_lines_w_cons.merge(feeder_reliability_2020,how='left',on='Feeder_Nam')
meter_lines_w_cons_w_reliabilty = meter_lines_w_cons_w_reliabilty.merge(feeder_reliability_2019,how='left',on='Feeder_Nam')
meter_lines_w_cons_w_reliabilty = meter_lines_w_cons_w_reliabilty.merge(feeder_reliability_2018,how='left',on='Feeder_Nam')
meter_lines_w_cons_w_reliabilty = meter_lines_w_cons_w_reliabilty.merge(feeder_reliability_2017,how='left',on='Feeder_Nam')
meter_lines_w_cons_w_reliabilty = meter_lines_w_cons_w_reliabilty.merge(feeder_reliability_2016,how='left',on='Feeder_Nam')

/rigel/dsi/users/jm5352/miniconda3/envs/nenv/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/rigel/dsi/users/jm5352/miniconda3/envs/nenv/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


CPU times: user 2min 48s, sys: 5.33 s, total: 2min 53s
Wall time: 2min 56s


In [3]:
%%time

# Merge meta data (customer descriptive info) with customer consumption data
customer_consumption  = meta_df[['meter_serial_number_consumer_id','installation_date']].merge(consumption[
    ['meter_serial_number_consumer_id','Meter_Numb','month','year','kWhs']], how='left', on='meter_serial_number_consumer_id')
customer_consumption['installation_date'] = pd.to_datetime(customer_consumption.installation_date, format='%Y-%m-%d')

# drop Rows without meter ids
customer_consumption = customer_consumption[~customer_consumption.Meter_Numb.isna()]

# Minimum number of consumption months in calendar year
min_threshold = 6

# Algorithm to compute annual consumption per meter
install_start_year = 1996
install_end_years   = [2012, 2013, 2014, 2015, 2016, 2017, 2018]
consumption_years   = [2013, 2014, 2015, 2016, 2017, 2018, 2019]

# Compute average mature meter/customer consumption
all_annual_consumption = pd.DataFrame([])
for i in range(len(consumption_years)):
    # select install year range
    install_years = customer_consumption[
        (customer_consumption.installation_date>= pd.to_datetime(f'{install_start_year}-01-1') 
         )&(customer_consumption.installation_date<pd.to_datetime(f'{install_end_years[i]}-01-1'))]

    # compute mature consumption, consumption a year after connection
    mature_consumption = install_years[install_years.year==consumption_years[i]]
    
    # select meter ids that have atleast six months of consumption in a calendar year
    Number_of_consumption_months = mature_consumption.groupby(['meter_serial_number_consumer_id'])['month'].count().reset_index().rename(columns={'month':'month_count'})
    meter_serial_number_consumer_id_above_min_threshold = Number_of_consumption_months[Number_of_consumption_months.month_count>=min_threshold]
    meter_serial_number_consumer_id_above_min_threshold_list = meter_serial_number_consumer_id_above_min_threshold.meter_serial_number_consumer_id.tolist()
    
    # filter out meter ids with less than six months consumption
    mature_consumption = mature_consumption[mature_consumption.meter_serial_number_consumer_id.isin(meter_serial_number_consumer_id_above_min_threshold_list)]
    
    # compute customer annual consumption in target year
    annual_mature_consumption = mature_consumption.groupby(
        ['meter_serial_number_consumer_id'])['kWhs'].sum().reset_index().rename(columns={'kWhs':'annual_kWhs'}) # use meter_serial_number_consumer_id to compute annual consumption
    annual_mature_consumption['consumption_year'] = consumption_years[i] # add correponding consumption year
    all_annual_consumption = pd.concat([all_annual_consumption, annual_mature_consumption], ignore_index=True) # concat in a dataframe for all consumption years

# add installation dates and customer types
all_annual_consumption_with_install_dates = all_annual_consumption.merge(
    meta_df[['meter_serial_number_consumer_id', 'Meter_Numb','installation_date','eguide_categories']], on='meter_serial_number_consumer_id', how='left')

# create pivot table
all_annual_consumption_with_install_dates_with_install_dates_pvt = all_annual_consumption_with_install_dates.pivot(index=['meter_serial_number_consumer_id','Meter_Numb','installation_date','eguide_categories'],
                                                                                      columns='consumption_year', values='annual_kWhs').reset_index()
# remove duplicates
annual_consumption_no_dups = all_annual_consumption_with_install_dates_with_install_dates_pvt[~(all_annual_consumption_with_install_dates_with_install_dates_pvt[
                                                                            ['Meter_Numb']].duplicated(keep='first'))]


# merge reliablity with consumption
reliability_w_consumption = meter_lines_w_cons_w_reliabilty.merge(annual_consumption_no_dups[['Meter_Numb',2019]],
                                      how='left', on='Meter_Numb').rename(columns={2019:'annual_consumption_2019'})
reliability_w_consumption_no_dups = reliability_w_consumption[~(reliability_w_consumption.Meter_Numb.duplicated(keep='first'))]




CPU times: user 36.6 s, sys: 8.75 s, total: 45.3 s
Wall time: 45.4 s


## consumption in 2019

In [4]:
customer_consumption_annual = customer_consumption[customer_consumption.year==2019].copy()

# add last add of month as end of monthly transaction period
customer_consumption_annual['trans_period'] = pd.to_datetime(customer_consumption_annual[['year', 'month']].assign(day=lambda x: x['month'].apply(
    lambda y: 31 if y in [1, 3, 5, 7, 8, 10, 12] else (30 if y in [4, 6, 9, 11] else (28 if y == 2 else np.nan)))))

customer_consumption_annual['days_since_connection'] = ((customer_consumption_annual['trans_period'] - customer_consumption_annual['installation_date']).dt.days).astype(int)
# replace negative days with zero
customer_consumption_annual.days_since_connection[customer_consumption_annual.days_since_connection<0] = 0

# groupby meter ID
customer_consumption_annual_gpby = customer_consumption_annual.groupby('Meter_Numb')['days_since_connection'].max().reset_index()

# merge
reliabilty_cons_w_conn_days = reliability_w_consumption_no_dups.merge(customer_consumption_annual_gpby, how='left', on='Meter_Numb')

/tmp/ipykernel_4201/2028390809.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  customer_consumption_annual.days_since_connection[customer_consumption_annual.days_since_connection<0] = 0


### save dataframe

In [ ]:
%%time
pck_fp = './data/processed/reliabilty_cons_w_conn_days_22_11_22_12am.pck'
reliabilty_cons_w_conn_days.to_pickle(pck_fp)